# Reporte Kpis clutch

In [ ]:
import os

from_drive = True  # same flag you use everywhere

if os.environ.get("ATLAS_BOOTSTRAPPED") != "1":
    # ---------- GIT ON COLAB ONLY ----------
    try:
        from google.colab import userdata

        git_token = userdata.get('gitToken')
        git_user = userdata.get('gitUser')
        git_url = f'https://{git_token}@github.com/rene-aum/Atlas.git'
        branch_to_pull = 'dev'

        os.chdir('/content')

        if not os.path.isdir('Atlas'):
            !git clone {git_url}

        %cd Atlas
        !git fetch origin {branch_to_pull}
        !git checkout {branch_to_pull}
        !git pull origin {branch_to_pull}

        !pip install -r PipelinesConsumo/src/requirements.txt
        %cd PipelinesConsumo

    except Exception as e:
        print(e)
        print('Running in other environment not colab probably!')

    # ---------- DRIVE + SHEETS ----------
    if from_drive:
        from pydrive2.auth import GoogleAuth
        from pydrive2.drive import GoogleDrive
        from google.colab import auth
        from oauth2client.client import GoogleCredentials
        import gspread
        from google.auth import default
        from gspread_dataframe import set_with_dataframe
        import gdown

        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        drive = GoogleDrive(gauth)

        creds, _ = default()
        gc = gspread.authorize(creds)

    os.environ["ATLAS_BOOTSTRAPPED"] = "1"
else:
    print("Bootstrap already done, assuming orchestrator ran it.")

In [ ]:
import sys
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
import sys
sys.path.append('..')
sys.path.append('../..')
from utils.utils import (get_dates_dataframe,
                       add_year_week,
                       custom_read,
                       process_columns,
                       remove_accents)
from PipelinesConsumo.src.rawAtlas import RawAtlas
from PipelinesConsumo.src.processedAtlas import ProcessedAtlas
from src.transformed import Transformed
from utils.drive_toolbox import(from_drive_to_local,
                             get_last_modification_date_drive,
                             create_sheets_in_drive_folder,
                             update_sheets_in_drive_folder,
                             read_from_google_sheets,
                             list_file_ids_for_drive_folder,
                             create_csv_file_in_drive_folder,
                             write_csv_to_drive,
                             read_csv_from_drive)
from src.constants import (atlas_raw_output_folder_id,
                           atlas_consumo_output_folder_id,
                           consumo_sheets_ids_dict,
                           data_source_folder_id,
                           raw_output_ids,
                           folder_id_bauto_gabo,
                           id_reporte_ventas,
                           id_edas_referenciados,
                           id_torre_de_control
                           )


warnings.filterwarnings('ignore')



## Load

In [ ]:

torreRaw = read_from_google_sheets(gc,id_torre_de_control,sheetname='Asignación compradores')
torreRaw.columns

subset_columns = ['id_lead','origen_automarket','id_comprador','id_de_ultimo_pedido','folio_bauto',
                  'espacio_automarket','asesor_de_ventas','fecha_de_asignacion','total_apartados',
                  'estatus_de_lead','lead_contactado_(visualiza_cc)',
                  'fecha_de_cierre_del_lead','motivos_de_cancelacion',
                  'correo_recibido_en_buzon_contingencia','documentacion_completa_contingencia',
                  ]
rename_dict = {'lead_contactado_(visualiza_cc)':'lead_contactado'}
torre = (torreRaw
.pipe(process_columns)
[subset_columns]
 .assign(folio = lambda x: pd.to_numeric(x['folio_bauto'],errors='coerce').astype('Int64'),
         id_comprador = lambda x: pd.to_numeric(x['id_comprador'],errors='coerce').astype('Int64'),
         fecha_de_asignacion = lambda x: pd.to_datetime(x['fecha_de_asignacion'],format='%d/%m/%Y').dt.strftime('%Y-%m-%d'),
        #  fecha_de_cierre_del_lead = lambda x: pd.to_datetime(x['fecha_de_cierre_del_lead'],format='%d/%m/%Y').dt.strftime('%Y-%m-%d'),
         )
 .rename(columns=rename_dict)
 .sort_values(by='fecha_de_asignacion',ascending=False)
 )
